In [69]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
from collections import defaultdict
from TorchCRF import CRF


# Define parameters
MAX_SEQUENCE_LENGTH = 25  # Adjust as needed
EMBEDDING_DIM = 100
VOCAB_SIZE = 8000
HIDDEN_DIM = 64
BATCH_SIZE = 16
EPOCHS = 4

# Labels and mapping
LABELS = ['O', 'B-Task', 'I-Task', 'B-Date', 'I-Date', 'B-Time', 'I-Time']
NUM_CLASSES = len(LABELS)
label2idx = {label: idx for idx, label in enumerate(LABELS)}
idx2label = {idx: label for label, idx in label2idx.items()}

In [70]:
df = pd.read_csv('NER_Data.csv')
df.head(10)

,Task,Label
0,check tickets for the gathering on may 14th at...,"B-Task,I-Task,I-Task,O,B-Task,O,B-Date,I-Date,..."
1,confirm a architect to inspect the book by tom...,"B-Task,O,B-Task,I-Task,I-Task,I-Task,I-Task,O,..."
2,finish reading chapter 60 of tech conference b...,"B-Task,I-Task,I-Task,I-Task,I-Task,I-Task,I-Ta..."
3,Remember to confirm with Gal about the trainin...,"O,O,B-Task,I-Task,I-Task,I-Task,I-Task,I-Task,..."
4,call a garden follow-up by the 18th in 14,"B-Task,O,B-Task,I-Task,O,B-Date,I-Date,O,B-Time"
5,check Lia about the client presentation by nex...,"B-Task,I-Task,I-Task,I-Task,I-Task,I-Task,O,B-..."
6,test preliminary idea report by 3:15 a.m. on W...,"B-Task,I-Task,I-Task,I-Task,O,B-Time,I-Time,O,..."
7,analyze the book tomorrow,"B-Task,O,B-Task,B-Date"
8,prepare the gym bag table to Daniel by next Tu...,"B-Task,O,B-Task,I-Task,I-Task,I-Task,I-Task,O,..."
9,prepare the project report by 4AM on Wednesday,"B-Task,O,B-Task,I-Task,O,B-Time,O,B-Date"


In [ ]:
# Example training data
task_examples = df['Task'].astype(str)
print(task_examples[:3])

y_train = df['Label'].apply(lambda x: x.split(","))

# Vocabulary
vocab = defaultdict(lambda: 1)  # Unknown words map to index 1
vocab.update({word.strip().lower(): idx+2 for idx, word in enumerate(set(" ".join(task_examples).split()))})  # Start from index 2

# Process data
X_train = [[vocab[word] for word in example.split()] for example in task_examples]
X_train_padded = [seq + [0] * (MAX_SEQUENCE_LENGTH - len(seq)) for seq in X_train]  # Pad to max length
y_train_indices = [[label2idx.get(label, 0) for label in sent] for sent in y_train]
y_train_padded = [seq + [0] * (MAX_SEQUENCE_LENGTH - len(seq)) for seq in y_train_indices]

0    check tickets for the gathering on may 14th at...
1    confirm a architect to inspect the book by tom...
2    finish reading chapter 60 of tech conference b...
Name: Task, dtype: object


In [ ]:
X_train_padded, X_test, y_train_padded, y_test = train_test_split(X_train_padded, y_train_padded, test_size=0.2, random_state=1)

In [ ]:
import pickle

# Save vocabulary as regular dictionary
with open('NERVocabulary.pkl', 'wb') as f:
    pickle.dump(dict(vocab), f)

# Save label mappings
with open('label_mappings.pkl', 'wb') as f:
    pickle.dump({'label2idx': label2idx, 'idx2label': idx2label}, f)

In [74]:
# PyTorch Dataset class
class TaskDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.long)
        self.y = torch.tensor(y, dtype=torch.long)
    
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

dataset = TaskDataset(X_train_padded, y_train_padded)
dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)

In [75]:
# Model definition
class BiLSTM_NER(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, num_classes):
        super(BiLSTM_NER, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True, bidirectional=True)
        self.dropout = nn.Dropout(0.3)
        self.fc = nn.Linear(hidden_dim * 2, num_classes)  # BiLSTM doubles hidden size
        self.crf = CRF(num_classes)

    def forward(self, x, tags=None, mask=None):
        x = self.embedding(x)
        x, _ = self.lstm(x)
        x = self.dropout(x)
        emissions = self.fc(x)

        if tags is not None:  # Training
            loss = -self.crf(emissions, tags, mask=mask)
            return loss
        else:  # Prediction
            return self.crf.viterbi_decode(emissions, mask=mask)

In [76]:
# Instantiate model
model = BiLSTM_NER(VOCAB_SIZE, EMBEDDING_DIM, HIDDEN_DIM, NUM_CLASSES)
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
def train_model(model, dataloader, epochs):
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        for X_batch, y_batch in dataloader:
            mask = (X_batch != 0)  # Mask for padded tokens - True where not padded
            optimizer.zero_grad()
            loss = model(X_batch, y_batch, mask)  # CRF loss
            # Ensure loss is a scalar
            if loss.dim() > 0:  # If loss is a tensor (one per sequence)
                loss = loss.mean()  # Reduce to scalar by averaging over batch
            loss.backward()  # Backpropagate
            optimizer.step()
            total_loss += loss.item()
        avg_loss = total_loss / len(dataloader)
        print(f"Epoch {epoch+1}/{epochs}, Avg Loss: {avg_loss:.4f}")
    print("Training complete!")

train_model(model, dataloader, EPOCHS)

Epoch 1/4, Avg Loss: 2.0703
Epoch 2/4, Avg Loss: 0.0285
Epoch 3/4, Avg Loss: 0.0085
Epoch 4/4, Avg Loss: 0.0041
Training complete!


In [ ]:
def predict(sentence):
    model.eval()
    tokens = [vocab[word.strip().lower()] for word in sentence.split()]
    padded = tokens + [0] * (MAX_SEQUENCE_LENGTH - len(tokens))
    input_tensor = torch.tensor([padded], dtype=torch.long)
    mask = (input_tensor != 0)
    with torch.no_grad():
        preds = model(input_tensor, mask=mask)[0]  # CRF decode returns list
    return [idx2label[idx] for idx in preds[:len(tokens)]]

In [79]:
# Test predictions
test_sentences = [
    "practice presentation slides for the client pitch on June 15th at two pm",
    "on Oct 22nd at 17 Review the project timeline document",
    "Book a rideshare for the airport which is on Thursday by tomorrow at 20:00"
]
for sentence in test_sentences:
    pred = predict(sentence)
    print(f"\nSentence: {sentence}")
    print(f"Predicted: {pred}")


Sentence: practice presentation slides for the client pitch on June 15th at two pm
Predicted: ['B-Task', 'I-Task', 'I-Task', 'I-Task', 'O', 'B-Task', 'I-Task', 'O', 'B-Date', 'I-Date', 'O', 'B-Time', 'I-Time']

Sentence: on Oct 22nd at 17 Review the project timeline document
Predicted: ['O', 'B-Date', 'I-Date', 'O', 'B-Time', 'B-Task', 'O', 'B-Task', 'I-Task', 'I-Task']

Sentence: Book a rideshare for the airport which is on Thursday by tomorrow at 20:00
Predicted: ['B-Task', 'I-Task', 'I-Task', 'I-Task', 'O', 'B-Task', 'O', 'O', 'O', 'O', 'O', 'B-Date', 'O', 'B-Time']


In [ ]:
# Check CRF transition matrix
print(model.crf.trans_matrix)

Parameter containing:
tensor([[-0.0209,  0.1925, -0.2715,  0.1916, -0.3821,  0.1330, -0.2418],
        [-0.0467, -0.2933,  0.2943,  0.0059, -0.0413, -0.1463, -0.1064],
        [ 0.1743, -0.3135,  0.2408, -0.0878, -0.1855, -0.2330,  0.0017],
        [-0.0941, -0.0898, -0.1595, -0.2542,  0.1967, -0.0070, -0.1066],
        [ 0.2417,  0.0167, -0.1076, -0.1131,  0.0923, -0.1108, -0.0908],
        [-0.1717,  0.0189, -0.1386, -0.2212, -0.2889, -0.2752,  0.2809],
        [ 0.0074, -0.0427, -0.0256,  0.0757, -0.1718,  0.0136, -0.0348]],
       requires_grad=True)


In [81]:
torch.save(model.state_dict(), 'NERModel.pth')

In [ ]:
from seqeval.metrics import classification_report, f1_score

def evaluate(model, X_data, y_data, idx2label):
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for x, y_true in zip(X_data, y_data):
            mask = [token != 0 for token in x]
            input_tensor = torch.tensor([x], dtype=torch.long)
            mask_tensor = torch.tensor([mask], dtype=torch.bool)
            preds = model(input_tensor, mask=mask_tensor)[0]  # Viterbi output

            true_labels = [idx2label[idx] for idx, m in zip(y_true, mask) if m]
            pred_labels = [idx2label[idx] for idx, m in zip(preds, mask) if m]

            all_labels.append(true_labels)
            all_preds.append(pred_labels)

    print(classification_report(all_labels, all_preds))
    print("F1-score:", f1_score(all_labels, all_preds))

evaluate(model, X_test, y_test, idx2label)

              precision    recall  f1-score   support

        Date       1.00      1.00      1.00      3000
        Task       1.00      1.00      1.00      5305
        Time       1.00      1.00      1.00      2303

   micro avg       1.00      1.00      1.00     10608
   macro avg       1.00      1.00      1.00     10608
weighted avg       1.00      1.00      1.00     10608

F1-score: 1.0


In [ ]:
test_samples = [
    {
        "sentence": "Send the updated budget report to Sarah by next Wednesday at 9am",
        "labels": "B-Task,O,B-Task,I-Task,I-Task,I-Task,I-Task,O,B-Date,I-Date,O,B-Time"
    },
    {
        "sentence": "Schedule a dentist appointment on the 10th around noon",
        "labels": "B-Task,O,B-Task,I-Task,O,B-Date,I-Date,O,B-Time"
    },
    {
        "sentence": "Check inventory levels for the Tel Aviv branch tomorrow noon",
        "labels": "B-Task,I-Task,I-Task,I-Task,O,B-Task,I-Task,I-Task,B-Date,B-Time"
    },
    {
        "sentence": "Finalize the UI mockups before the meeting on Friday at 14:00",
        "labels": "B-Task,O,B-Task,I-Task,O,O,O,O,B-Date,O,B-Time"
    },
    {
        "sentence": "Organize carpool for the school event this Sunday at ten",
        "labels": "B-Task,I-Task,I-Task,O,B-Task,I-Task,O,B-Date,O,B-Time"
    },
    {
        "sentence": "Prepare the slides and notes by April 3rd at 11:30 AM",
        "labels": "B-Task,O,B-Task,I-Task,I-Task,O,B-Date,I-Date,O,B-Time,I-Time"
    },
    {
        "sentence": "Email the new interns about onboarding next Tuesday",
        "labels": "B-Task,O,B-Task,I-Task,I-Task,I-Task,B-Date,I-Date"
    },
    {
        "sentence": "Remind John to submit the forms by midnight on the 5th",
        "labels": "B-Task,I-Task,I-Task,I-Task,I-Task,I-Task,O,B-Time,B-Date,I-Date,I-Date"
    },
    {
        "sentence": "Print handouts for the seminar scheduled at noon on September 1st",
        "labels": "B-Task,I-Task,I-Task,O,B-Task,O,O,B-Time,O,B-Date,I-Date"
    },
    {
        "sentence": "Plan the team lunch in Herzliya after 13:00 this Friday",
        "labels": "B-Task,O,B-Task,I-Task,I-Task,I-Task,O,B-Time,O,B-Date"
    }
]

from seqeval.metrics import classification_report

true_labels = []
pred_labels = []

for sample in test_samples:
    sentence = sample["sentence"]
    gold = sample["labels"].split(",")
    pred = predict(sentence)
    print(f"\nSentence: {sentence}")
    print(f"Prediction: {pred}")
    print(f"True:       {gold}")
    
    true_labels.append(gold)
    pred_labels.append(pred)

print("\nEvaluation Report:")
print(classification_report(true_labels, pred_labels))


Sentence: Send the updated budget report to Sarah by next Wednesday at 9am
Prediction: ['B-Task', 'O', 'B-Task', 'I-Task', 'I-Task', 'I-Task', 'I-Task', 'O', 'B-Date', 'I-Date', 'O', 'B-Time']
True:       ['B-Task', 'O', 'B-Task', 'I-Task', 'I-Task', 'I-Task', 'I-Task', 'O', 'B-Date', 'I-Date', 'O', 'B-Time']

Sentence: Schedule a dentist appointment on the 10th around noon
Prediction: ['B-Task', 'O', 'B-Task', 'I-Task', 'O', 'B-Date', 'I-Date', 'O', 'B-Time']
True:       ['B-Task', 'O', 'B-Task', 'I-Task', 'O', 'B-Date', 'I-Date', 'O', 'B-Time']

Sentence: Check inventory levels for the Tel Aviv branch tomorrow noon
Prediction: ['B-Task', 'I-Task', 'I-Task', 'I-Task', 'O', 'B-Task', 'I-Task', 'I-Task', 'B-Date', 'B-Time']
True:       ['B-Task', 'I-Task', 'I-Task', 'I-Task', 'O', 'B-Task', 'I-Task', 'I-Task', 'B-Date', 'B-Time']

Sentence: Finalize the UI mockups before the meeting on Friday at 14:00
Prediction: ['B-Task', 'I-Task', 'I-Task', 'I-Task', 'I-Task', 'O', 'O', 'O', 'B-Date